## import


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, confusion_matrix
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier


## Load Data

In [2]:
# Load data
df = pd.read_csv("../data/processed/train_final.csv")

# Columns to drop
drop_cols = [
    "PotentialFraud", 
    "Provider"

]

# Features / target
X = df.drop(columns=drop_cols)
y = df["PotentialFraud"].astype(int)

# Train / validation / test split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

## basic Evaluation to choose a model

In [3]:
def find_threshold_for_recall(y_true, y_probs, target_recall=0.90):
    prec, rec, thresh = precision_recall_curve(y_true, y_probs)
    
    # recall is in descending order, find highest threshold where recall >= target
    valid_idx = np.where(rec >= target_recall)[0]
    
    if len(valid_idx) == 0:
        return 0.0
    
    # get the last valid index (highest threshold with recall >= target)
    # thresh has one fewer element than prec/rec, so cap at len(thresh)-1
    best_idx = min(valid_idx[-1], len(thresh) - 1)
    return thresh[best_idx]

In [4]:
def evaluate_with_threshold(name, y_true, y_probs, threshold):
    """Evaluate model at a specific threshold, print metrics, and return a summary dict."""
    y_pred = (y_probs >= threshold).astype(int)
    
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_probs)
    avg_prec = average_precision_score(y_true, y_probs)
    cm = confusion_matrix(y_true, y_pred)
    
    result = {
        "model": name,
        "threshold": float(threshold),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "roc_auc": float(roc_auc),
        "avg_precision": float(avg_prec),
        "confusion_matrix": cm.tolist()
    }
    
    print(f"{name} @ threshold={threshold:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ROC AUC: {roc_auc:.4f}, AP: {avg_prec:.4f}")
    print("Confusion matrix:")
    print(cm)
    
    return result

In [5]:
def best_precision_with_minimum_recall(y_true, prob, target_recall=0.92, steps=2000):
    thresholds = np.linspace(0, 1, steps)
    best_thresh, best_prec, best_rec = 0.0, 0.0, 0.0
    for t in thresholds:
        preds = (prob >= t).astype(int)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds)
        if rec >= target_recall and prec > best_prec:
            best_prec, best_rec, best_thresh = prec, rec, t
    return best_thresh, best_prec, best_rec

## Gradient Boost

In [6]:
# ============================================================
# HANDLE IMBALANCE
# ============================================================
scale = (y_train == 0).sum() / (y_train == 1).sum()
sample_weights = np.where(y_train == 1, scale, 1)
print("Class imbalance ratio:", scale)

# ============================================================
# SAFE HYPERPARAMETER GRID (prevents overfitting)
# ============================================================
gb_base = GradientBoostingClassifier(random_state=42)

param_grid_gb = {
    "n_estimators": [150, 200, 250],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.03, 0.05],
    "subsample": [0.7, 0.8],
    "min_samples_leaf": [1, 2],
    "min_samples_split": [2, 4]
}

grid_gb = GridSearchCV(
    gb_base,
    param_grid_gb,
    scoring="average_precision",   # BEST for fraud
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_gb.fit(X_train, y_train, sample_weight=sample_weights)

print("\nBest GB params:", grid_gb.best_params_)
print("Best CV AP:", grid_gb.best_score_)

gb_best = grid_gb.best_estimator_

# ============================================================
# CALIBRATION
# ============================================================
gb_cal = CalibratedClassifierCV(gb_best, method="isotonic", cv=3)
gb_cal.fit(X_train, y_train, sample_weight=sample_weights)

gb_probs = gb_cal.predict_proba(X_val)[:, 1]

# ============================================================
# BEST PRECISION while recall ≥ 0.92
# ============================================================
def best_precision_with_minimum_recall(y_true, prob, target_recall=0.92):
    thresholds = np.linspace(0, 1, 2000)
    best_thresh, best_prec, best_rec = 0, 0, 0

    for t in thresholds:
        preds = (prob >= t).astype(int)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds)

        if rec >= target_recall and prec > best_prec:
            best_prec = prec
            best_rec = rec
            best_thresh = t

    return best_thresh, best_prec, best_rec


thresh, best_prec, best_rec = best_precision_with_minimum_recall(
    y_val, gb_probs, target_recall=0.92
)

print("\nOptimal Threshold:", thresh)
print("Precision:", best_prec)
print("Recall:", best_rec)

gb_result = evaluate_with_threshold("GB Calibrated Optimized", y_val, gb_probs, thresh)
gb_result


Class imbalance ratio: 9.686419753086419
Fitting 3 folds for each of 144 candidates, totalling 432 fits

Best GB params: {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250, 'subsample': 0.8}
Best CV AP: 0.9440819180776305

Best GB params: {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250, 'subsample': 0.8}
Best CV AP: 0.9440819180776305

Optimal Threshold: 0.6208104052026012
Precision: 0.5340909090909091
Recall: 0.9215686274509803
GB Calibrated Optimized @ threshold=0.6208
Precision: 0.5341, Recall: 0.9216, F1: 0.6763, ROC AUC: 0.9661, AP: 0.7071
Confusion matrix:
[[449  41]
 [  4  47]]

Optimal Threshold: 0.6208104052026012
Precision: 0.5340909090909091
Recall: 0.9215686274509803
GB Calibrated Optimized @ threshold=0.6208
Precision: 0.5341, Recall: 0.9216, F1: 0.6763, ROC AUC: 0.9661, AP: 0.7071
Confusion matrix:
[[449  41]
 [  4  47]]


{'model': 'GB Calibrated Optimized',
 'threshold': 0.6208104052026012,
 'precision': 0.5340909090909091,
 'recall': 0.9215686274509803,
 'f1': 0.6762589928057554,
 'roc_auc': 0.9660864345738296,
 'avg_precision': 0.707050604082991,
 'confusion_matrix': [[449, 41], [4, 47]]}

## XGBoost

In [7]:
# imbalance handling
scale = (y_train == 0).sum() / (y_train == 1).sum()
sample_weights = np.where(y_train == 1, scale, 1)
print(f"Class imbalance ratio (XGB): {scale:.2f}")

xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

param_grid_xgb = {
    "n_estimators": [200, 400],
    "max_depth": [4, 5, 6],
    "learning_rate": [0.03, 0.05],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.7, 0.8]
}

grid_xgb = GridSearchCV(
    xgb_base,
    param_grid_xgb,
    scoring="average_precision",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train, y_train, sample_weight=sample_weights)

print(f"\nBest XGB params: {grid_xgb.best_params_}")
print(f"Best XGB CV AP Score: {grid_xgb.best_score_:.4f}")

# Use best estimator directly
xgb_best = grid_xgb.best_estimator_

# Calibrate
xgb_cal = CalibratedClassifierCV(xgb_best, method="isotonic", cv=3)
xgb_cal.fit(X_train, y_train, sample_weight=sample_weights)

# Get probabilities
xgb_probs = xgb_cal.predict_proba(X_val)[:, 1]

# Find threshold for 92% recall
xgb_thresh, xgb_prec, xgb_rec = best_precision_with_minimum_recall(y_val, xgb_probs, target_recall=0.92)
print(f"\nXGB optimal threshold: {xgb_thresh:.4f}, precision: {xgb_prec:.4f}, recall: {xgb_rec:.4f}")

# Evaluate with threshold
xgb_result = evaluate_with_threshold("XGBoost (Calibrated)", y_val, xgb_probs, xgb_thresh)

xgb_result

Class imbalance ratio (XGB): 9.69
Fitting 3 folds for each of 48 candidates, totalling 144 fits

Best XGB params: {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.8}
Best XGB CV AP Score: 0.9442

Best XGB params: {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.8}
Best XGB CV AP Score: 0.9442

XGB optimal threshold: 0.5653, precision: 0.5402, recall: 0.9216
XGBoost (Calibrated) @ threshold=0.5653
Precision: 0.5402, Recall: 0.9216, F1: 0.6812, ROC AUC: 0.9635, AP: 0.6970
Confusion matrix:
[[450  40]
 [  4  47]]

XGB optimal threshold: 0.5653, precision: 0.5402, recall: 0.9216
XGBoost (Calibrated) @ threshold=0.5653
Precision: 0.5402, Recall: 0.9216, F1: 0.6812, ROC AUC: 0.9635, AP: 0.6970
Confusion matrix:
[[450  40]
 [  4  47]]


{'model': 'XGBoost (Calibrated)',
 'threshold': 0.5652826413206603,
 'precision': 0.5402298850574713,
 'recall': 0.9215686274509803,
 'f1': 0.6811594202898551,
 'roc_auc': 0.9635054021608643,
 'avg_precision': 0.6969718380402277,
 'confusion_matrix': [[450, 40], [4, 47]]}

## Logistic Regression

In [8]:
# LogisticRegression: use class_weight & sample_weight
scale = (y_train == 0).sum() / (y_train == 1).sum()
sample_weights = np.where(y_train == 1, scale, 1)
print("Class imbalance ratio (LR):", round(scale,2))

log_base = LogisticRegression(max_iter=5000, random_state=42, solver='liblinear')

param_grid_log = {
    "C": [0.01, 0.1, 1.0, 10.0],
    "penalty": ["l2"]
}

grid_log = GridSearchCV(
    log_base,
    param_grid_log,
    scoring="average_precision",
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_log.fit(X_train, y_train, sample_weight=sample_weights)

print("\nBest LR params:", grid_log.best_params_)
print("Best CV AP:", grid_log.best_score_)

log_best = grid_log.best_estimator_

log_cal = CalibratedClassifierCV(log_best, method="isotonic", cv=3)
log_cal.fit(X_train, y_train, sample_weight=sample_weights)

log_probs = log_cal.predict_proba(X_val)[:, 1]

log_thresh, log_prec, log_rec = best_precision_with_minimum_recall(y_val, log_probs, target_recall=0.92)
print("\nLR optimal threshold:", log_thresh, "precision:", log_prec, "recall:", log_rec)

evaluate_with_threshold("LogReg (Calibrated Optimized)", y_val, log_probs, log_thresh)


Class imbalance ratio (LR): 9.69
Fitting 3 folds for each of 4 candidates, totalling 12 fits

Best LR params: {'C': 10.0, 'penalty': 'l2'}
Best CV AP: 0.9350319277599715

Best LR params: {'C': 10.0, 'penalty': 'l2'}
Best CV AP: 0.9350319277599715

LR optimal threshold: 0.3091545772886443 precision: 0.30128205128205127 recall: 0.9215686274509803
LogReg (Calibrated Optimized) @ threshold=0.3092
Precision: 0.3013, Recall: 0.9216, F1: 0.4541, ROC AUC: 0.9459, AP: 0.7336
Confusion matrix:
[[381 109]
 [  4  47]]

LR optimal threshold: 0.3091545772886443 precision: 0.30128205128205127 recall: 0.9215686274509803
LogReg (Calibrated Optimized) @ threshold=0.3092
Precision: 0.3013, Recall: 0.9216, F1: 0.4541, ROC AUC: 0.9459, AP: 0.7336
Confusion matrix:
[[381 109]
 [  4  47]]


{'model': 'LogReg (Calibrated Optimized)',
 'threshold': 0.3091545772886443,
 'precision': 0.30128205128205127,
 'recall': 0.9215686274509803,
 'f1': 0.45410628019323673,
 'roc_auc': 0.9458783513405362,
 'avg_precision': 0.7336427296275101,
 'confusion_matrix': [[381, 109], [4, 47]]}

## RandomForest (Main model)

In [9]:
rf_base = RandomForestClassifier(
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

param_grid_rf = {
    "n_estimators": [200, 400],
    "max_depth": [8, 12, 16],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

grid_rf = GridSearchCV(
    rf_base,
    param_grid_rf,
    scoring="average_precision",
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X_train, y_train)

print(f"\nBest RF params: {grid_rf.best_params_}")
print(f"Best RF CV AP Score: {grid_rf.best_score_:.4f}")

# Use best estimator directly
rf_best = grid_rf.best_estimator_

# Calibrate
rf_cal = CalibratedClassifierCV(rf_best, method="isotonic", cv=3)
rf_cal.fit(X_train, y_train)

# Get probabilities
rf_probs = rf_cal.predict_proba(X_val)[:, 1]

# Find threshold for 90% recall
rf_threshold = find_threshold_for_recall(y_val, rf_probs, target_recall=0.90)
print(f"\nThreshold for 90% recall: {rf_threshold:.4f}")

# Evaluate with threshold
rf_result = evaluate_with_threshold("Random Forest (Calibrated)", y_val, rf_probs, rf_threshold)


rf_result

Fitting 5 folds for each of 24 candidates, totalling 120 fits

Best RF params: {'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best RF CV AP Score: 0.7217

Best RF params: {'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best RF CV AP Score: 0.7217

Threshold for 90% recall: 0.2014
Random Forest (Calibrated) @ threshold=0.2014
Precision: 0.5750, Recall: 0.9020, F1: 0.7023, ROC AUC: 0.9677, AP: 0.7381
Confusion matrix:
[[456  34]
 [  5  46]]

Threshold for 90% recall: 0.2014
Random Forest (Calibrated) @ threshold=0.2014
Precision: 0.5750, Recall: 0.9020, F1: 0.7023, ROC AUC: 0.9677, AP: 0.7381
Confusion matrix:
[[456  34]
 [  5  46]]


{'model': 'Random Forest (Calibrated)',
 'threshold': 0.20135756056808687,
 'precision': 0.575,
 'recall': 0.9019607843137255,
 'f1': 0.7022900763358778,
 'roc_auc': 0.9677470988395358,
 'avg_precision': 0.7381196814740336,
 'confusion_matrix': [[456, 34], [5, 46]]}

## Test data & prediction CSV

## FINAL MODEL: Random Forest with best params

In [10]:
print("=" * 50)
print("FINAL MODEL EVALUATION ON TEST SET")
print("=" * 50)
print(f"\nUsing threshold from validation: {rf_threshold:.4f}")

# Get test probabilities using the SAME model from validation
test_probs = rf_cal.predict_proba(X_test)[:, 1]

# Apply the validation threshold
test_preds = (test_probs >= rf_threshold).astype(int)

# Evaluate
test_result = evaluate_with_threshold("Random Forest (Final - Test)", y_test, test_probs, rf_threshold)

print("\n" + "=" * 50)
print("TEST SET RESULTS")
print("=" * 50)
print(f"Threshold (from validation): {rf_threshold:.4f}")
print(f"Test Precision: {test_result['precision']:.4f}")
print(f"Test Recall: {test_result['recall']:.4f}")
print(f"Test F1: {test_result['f1']:.4f}")
print(f"Test ROC AUC: {test_result['roc_auc']:.4f}")
print(f"Test AP: {test_result['avg_precision']:.4f}")

FINAL MODEL EVALUATION ON TEST SET

Using threshold from validation: 0.2014
Random Forest (Final - Test) @ threshold=0.2014
Precision: 0.5114, Recall: 0.9000, F1: 0.6522, ROC AUC: 0.9722, AP: 0.8370
Confusion matrix:
[[448  43]
 [  5  45]]

TEST SET RESULTS
Threshold (from validation): 0.2014
Test Precision: 0.5114
Test Recall: 0.9000
Test F1: 0.6522
Test ROC AUC: 0.9722
Test AP: 0.8370


In [11]:
# Saving models for evaluation
import os
import joblib

models_dir = os.path.abspath(os.path.join("..", "models"))
if not os.path.exists(models_dir):
    os.makedirs(models_dir, exist_ok=True)
print(f"Models will be saved to: {models_dir}")

# Save calibrated models
joblib.dump(gb_cal, os.path.join(models_dir, "gb_calibrated.joblib"))
print("Saved: gb_calibrated.joblib")

joblib.dump(xgb_cal, os.path.join(models_dir, "xgb_calibrated.joblib"))
print("Saved: xgb_calibrated.joblib")

joblib.dump(log_cal, os.path.join(models_dir, "log_calibrated.joblib"))
print("Saved: log_calibrated.joblib")

joblib.dump(rf_cal, os.path.join(models_dir, "rf_calibrated.joblib"))
print("Saved: rf_calibrated.joblib")

Models will be saved to: c:\Users\omarh\Fraud Detection\HealthCare-Insurance-Fraud-Detection\models
Saved: gb_calibrated.joblib
Saved: xgb_calibrated.joblib
Saved: log_calibrated.joblib
Saved: xgb_calibrated.joblib
Saved: log_calibrated.joblib
Saved: rf_calibrated.joblib
Saved: rf_calibrated.joblib


# LOAD EXTERNAL TEST DATA & GENERATE PREDICTIONS

In [12]:
# Load external test data (unlabeled - for submission)
test_df = pd.read_csv("../data/processed/test_final.csv")

# Keep Provider for output
providers = test_df["Provider"]

# Drop same columns as training
X_external = test_df.drop(columns=["Provider"])

# If PotentialFraud exists in test, drop it (for prediction)
if "PotentialFraud" in X_external.columns:
    X_external = X_external.drop(columns=["PotentialFraud"])

print(f"External test data shape: {X_external.shape}")
print(f"Training features: {X_train.shape[1]}")

# Ensure columns match training data
missing_cols = set(X_train.columns) - set(X_external.columns)
extra_cols = set(X_external.columns) - set(X_train.columns)

if missing_cols:
    print(f"Missing columns in test data: {missing_cols}")
if extra_cols:
    print(f"Extra columns in test data (will be dropped): {extra_cols}")
    X_external = X_external.drop(columns=list(extra_cols))

# Reorder columns to match training
X_external = X_external[X_train.columns]

# Generate predictions using rf_cal and rf_threshold
external_probs = rf_cal.predict_proba(X_external)[:, 1]
external_preds = (external_probs >= rf_threshold).astype(int)

print(f"\nPredictions generated for {len(external_preds)} providers")
print(f"Predicted Fraud: {external_preds.sum()} ({100*external_preds.mean():.2f}%)")
print(f"Predicted Non-Fraud: {(1-external_preds).sum()} ({100*(1-external_preds.mean()):.2f}%)")

External test data shape: (1353, 38)
Training features: 38

Predictions generated for 1353 providers
Predicted Fraud: 202 (14.93%)
Predicted Non-Fraud: 1151 (85.07%)


In [13]:
# Create submission dataframe
submission = pd.DataFrame({
    "Provider": providers,
    "PotentialFraud": external_preds,
    "FraudProbability": external_probs
})

# Save to CSV
output_path = "../data/predictions.csv"
submission.to_csv(output_path, index=False)

print(f"Predictions saved to: {output_path}")
print(f"\nSubmission shape: {submission.shape}")
print("\nSubmission preview:")
print(submission.head(10))

print("\n" + "=" * 50)
print("PREDICTION SUMMARY")
print("=" * 50)
print(f"Total Providers: {len(submission)}")
print(f"Predicted Fraudulent: {submission['PotentialFraud'].sum()}")
print(f"Predicted Non-Fraudulent: {(1 - submission['PotentialFraud']).sum()}")
print(f"Fraud Rate: {100 * submission['PotentialFraud'].mean():.2f}%")

Predictions saved to: ../data/predictions.csv

Submission shape: (1353, 3)

Submission preview:
   Provider  PotentialFraud  FraudProbability
0  PRV51002               0          0.004219
1  PRV51006               0          0.002362
2  PRV51009               0          0.029196
3  PRV51010               0          0.018108
4  PRV51018               0          0.012618
5  PRV51019               0          0.002362
6  PRV51020               0          0.022311
7  PRV51022               0          0.071145
8  PRV51028               0          0.002362
9  PRV51033               0          0.020796

PREDICTION SUMMARY
Total Providers: 1353
Predicted Fraudulent: 202
Predicted Non-Fraudulent: 1151
Fraud Rate: 14.93%
